# Runtime of Hard Thresholding Pursuit on Large Sensing Matrices

In [1]:
# Configure JAX for 64-bit computing
from jax.config import config
config.update("jax_enable_x64", True)

In [2]:
import jax
import jax.numpy as jnp
import cr.sparse as crs
import cr.sparse.dict as crdict
import cr.sparse.data as crdata
import cr.sparse.pursuit.htp as htp
import cr.sparse.ef as ef

In [3]:
M = 2560;
N = 10240;
K = 200;

In [4]:
Phi = crdict.gaussian_mtx(crs.KEYS[0], M, N)

In [5]:
Phi.shape

(2560, 10240)

In [6]:
a = 1;
b = 2;
x0, omega = crdata.sparse_biuniform_representations(crs.KEYS[1], a, b, N, K)

In [7]:
y0 = Phi @ x0

In [8]:
y0.shape

(2560,)

In [9]:
u_bound = crdict.upper_frame_bound(Phi)
step_size = float(0.98 / u_bound)

In [10]:
sol = htp.matrix_solve_jit(Phi, y0, K, step_size=step_size)

In [11]:
print(sol.iterations)

4


In [12]:
perf = ef.RecoveryPerformance(Phi, y0, x0, sol=sol)

In [13]:
perf.print()

M: 2560, N: 10240, K: 200
x_norm: 21.520, y_norm: 21.477
x_hat_norm: 21.520, h_norm: 8.02e-14, r_norm: 8.16e-14
recovery_snr: 288.57 dB, measurement_snr: 288.41 dB
x_dr: 6.00 dB, y_dr: 73.26 dB, x_hat_dr: 5.999 dB
T0: [   10    64   270   293   330   341   450   467   506   515   555   577
   670   702   806   824   938   963   974   982  1129  1130  1199  1209
  1230  1242  1271  1445  1518  1528  1598  1673  1710  1772  1898  1922
  2088  2100  2109  2170  2196  2286  2415  2476  2531  2654  2674  2777
  2820  2875  2897  2931  2938  2979  3018  3070  3109  3117  3141  3206
  3209  3221  3405  3430  3447  3565  3612  3672  3673  3711  3788  3828
  3880  3930  3965  4004  4028  4060  4161  4241  4277  4323  4412  4427
  4437  4442  4517  4564  4655  4720  4735  4874  4899  4900  4956  5061
  5127  5156  5197  5207  5246  5468  5512  5571  5582  5634  5694  5711
  5785  5794  5927  5959  6015  6035  6141  6151  6212  6252  6257  6260
  6363  6385  6437  6543  6691  6742  6812  6854  69

In [14]:
%timeit htp.matrix_solve_jit(Phi, y0, K, step_size=step_size).I.block_until_ready()

160 ms ± 61 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
